# Ionos server setup

## This document outlines the setup of the Virtual Private Server (VPS) on Ionos

1. The first steps are carried out via the Ionos administrator dashboard, which is accessed using the billing@sntech.co.uk account.

```{figure} IonosScreen1.png
```
2. Once you tell Ionos what type of VPS you want, they will provision it and then it will become visible in the Server link. I have installed an Ubuntu 20.04 server. There are a few good reasons for this. Linux is far more secure than a Windows server. Ubuntu 20.04 is a stable, long-term release. It is not as cutting edge, but that is a good thing. Not all software has been modified for the latest operating system versions.  
Ubuntu 20.04 ships with Python 3.8.5, which is preferable to 3.9 because many Python libraries including tensorflow are not available for 3.9 yet. However, we need to update a few elements which I will show later.

```{figure} IonosScreen2.png
```
3. There are some firewall modifications that need to be implemented. R Shiny uses port 3838, so we set the firewall to accept all incoming TCP connections

4. Most of the additional server options such as reverse proxies are set up using the Plesk desktop. But to run Plesk you need to enable the following ports to accept all TCP connections in the firewall:  
195.214.233.80:5224  
195.214.233.80:443  
195.214.233.81:5224  
195.214.233.81:443  
195.214.233.82:5224  
195.214.233.82:443  
Now you can access the Plesk desktop by pointing the browser to:  
https://77.68.125.129:8443  

## Setting up users
By default there is a superuser called 'root', which uses the Ionos password. As general advice in Unix systems **avoid doing anything as root**. A careless keystroke can irreparably destroy things.  

What we do instead is set up users with root privileges. For the most part these users can't screw up anything other than their own setup *unless* they prefix the command with **sudo**, which approximates as SuperUser DO.  

We first secure shell into the system (here, via Linux - I don't have ssh instructions for Windows):

```
ssh root@77.68.125.129

>Password: XXXXXXXXXX 
```

Now we add users. I've added myself, and a sntech user with root privileges. Note the **usermod -aG sudo** grants the user sudo privileges:

```
adduser craig
>craigspassword
usermod -aG sudo craig

adduser sntech
>sntechspassword
usermod -aG sudo sntech
```

Once we are done here, log out as root and back in as your username. The command is **su** or switch user:

```
su craig
```

```{warning}
Avoid logging in as root at all costs!
```

## Python
Python 3.8.5 is already installed (as is 2.7.18), but pip3 isn't. I use pip3 instead of anaconda to maintain Python packages. That's just personal preference, because I've had bad experiences with Anaconda.  
Update the system, and install pip and some other libraries that are useful:  

```
sudo apt-get update  
sudo apt install python3-pip  
sudo apt install -y build-essential libssl-dev libffi-dev python3-dev  
sudo apt install debian-keyring g++-multilib g++-9-multilib gcc-9-doc bzr libstdc++-9-doc  
sudo apt install -y python3-venv 
```

## MariaDB/MySQL
**Note:** Installation notes are on https://www.digitalocean.com/community/tutorials/how-to-install-mariadb-on-ubuntu-20-04  
MariaDB ships with Ubuntu 20.04, so there is no need to install it. However, on a new installation, MariaDB is completely open with no security, so we need to correct this.  
Run this to secure it:  

```
sudo mysql_secure_installation
```
There is no initial root password, so press \<enter\>.
You will be prompted to set the root password. **Note:** This should not be the same as any other password. The database password is not the same as a user password. They are separate entities.  

You will be prompted with a series of questions. Select Y for each question. This will remove the insecure initial options. Now quit mariadb.

Now we need to add regular users. First I will make an admin user with all privileges, and one for myself. Start mariadb as root:


```
sudo mariadb
GRANT ALL ON *.* TO 'admin'@'localhost' IDENTIFIED BY 'admindbpassword' WITH GRANT OPTION;
FLUSH PRIVILEGES;
exit

sudo mariadb
GRANT ALL ON *.* TO 'craig'@'localhost' IDENTIFIED BY 'craigdbpassword' WITH GRANT OPTION;
FLUSH PRIVILEGES;
exit
```

Now we will run a few status checks. If for some reason MariaDB is not running, we can start it.

```
#Check status
sudo systemctl status mariadb

#Should be running fine. If not:
#sudo systemctl start mariadb

# Extra checks...
sudo mysqladmin version

# Check the admin password authentication
mysqladmin -u admin -p version
```

This completes the MariaDB installation process


## R and Shiny
R is not installed by default, so we have to grab it. There are a few steps to ensure we have core libraries, but also allow the user to install libraries that R studio decided we didn't need.

```
sudo apt-get install r-base-dev

sudo apt-get update
sudo apt-get upgrade

#R Shiny

# update indices
sudo apt update -qq
# install two helper packages we need
sudo apt install --no-install-recommends software-properties-common dirmngr
# import the signing key (by Michael Rutter) for these repos
sudo apt-key adv --keyserver keyserver.ubuntu.com --recv-keys E298A3A825C0D65DFD57CBB651716619E084DAB9
# add the R 4.0 repo from CRAN -- adjust 'focal' to 'groovy' or 'bionic' as needed
sudo add-apt-repository "deb https://cloud.r-project.org/bin/linux/ubuntu $(lsb_release -cs)-cran40/"

sudo apt install --no-install-recommends r-base

#Add 5000+ Cran repositories
sudo add-apt-repository ppa:c2d4u.team/c2d4u4.0+

sudo apt install --no-install-recommends r-cran-rstan 

```

Now onto the Shiny server itself. Note Ubuntu 14 does indeed seem to be the latest version:

```
sudo su - -c "R -e \"install.packages('shiny', repos='https://cran.rstudio.com/')\""

sudo apt-get install gdebi-core
sudo wget https://download3.rstudio.org/ubuntu-14.04/x86_64/shiny-server-1.5.16.958-amd64.deb
sudo gdebi shiny-server-1.5.16.958-amd64.deb
```

This server setup takes care of all of the restarting when down and all the other issues. It is probably the most painless way of getting things up and running.  

Check status:

```
systemctl status shiny-server
```

Now, point a browser to: http://77.68.125.129:3838/sample-apps/hello  
You should see the *hello world* version of a Shiny program. Remember, the firewall must be set so port 3838 will accept all incoming TCP connections, so if you see nothing - check that.  

## Shiny App maintenance
The best approach to maintain Shiny Apps is by using Github. This link: https://deanattali.com/2015/05/09/setup-rstudio-shiny-server-digital-ocean/#shiny-git is written for DigitalOcean, and is pretty good.  

Using github makes commits and updates easier, without having to login and FTP.  

I have a github account called **sntech-analytics**.

First, we set up a github link on the server. Note the '.' after the chown is deliberate. But I don't think the statements are necessary, so I commented them out.  

```
ssh craig@77.68.125.129
craigspassword

sudo groupadd shiny-apps

sudo usermod -aG shiny-apps craig
sudo usermod -aG shiny-apps sntech
#This user is required. It is the default shiny user
sudo usermod -aG shiny-apps shiny
cd /srv/shiny-server
#sudo chown -R craig:shiny-apps .
#sudo chown -R shiny:shiny-apps .

sudo chmod g+w .
sudo chmod g+s .

sudo apt-get -y install git
git config --global user.email "craig@sntech.co.uk"
git config --global user.name "Craig Syms"
git config --global user.email "craig@sntech.co.uk"
git config --global user.name "SNTech"

cd /srv/shiny-server
git init

# This initalizes everything and should add the Shiny test Apps to the sntech git
git remote add origin https://github.com/sntech-analytics/shiny-server.git
git add .
git commit -m "Initial commit"
git push -u origin master

#When prompted for username...
sntech-analytics
sntechgitpassword
```

Shiny Apps are written by myself and committed to github. When I want to upload/update the apps to the server:  

```
ssh craig@77.68.125.129
craigspassword

git pull origin master
```

## Grafana
I couldn't get Grafana working properly on the server. It was set up OK, but on starting it was not listening on any port. There are a whole swag of options in the Grafana config file, so I thought 'screw it' and took another approach.  
Grafana is now being run via the Ubuntu docker image. This actually has a few benefits. Grafana's default security is not great, and a Docker image separates it from the rest of the file system. These are the steps to set up Docker & Grafana (notes at https://docs.docker.com/engine/install/ubuntu/):


```
sudo apt-get install \
    apt-transport-https \
    ca-certificates \
    curl \
    gnupg \
    lsb-release
```
```
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg
```
```
#NOTE: Next 3 lines are single command
echo \
  "deb [arch=amd64 signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/ubuntu \ 
  $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
```

To check the installation...  

```
sudo docker run hello-world
```

Now we can download and run the Grafana image.  

```
#First, set a physical storage location so you don't lose everything if the image restarts
sudo docker volume create grafana-storage
#Now the Docker image itself
sudo docker run -d -p 3000:3000 --name=grafana -v grafana-storage:/var/lib/grafana grafana/grafana
# Add this line so Grafana will restart if for some reason the server reboots
sudo docker update --restart unless-stopped grafana
```  

The default setup has the username *admin* with password *admin*. On a new installation, needless to say the first thing to do is change that.  

Grafana can now be accessed by pointing the browser to:  
http://77.68.125.129:3000/  
Note this is not a https port. There are some other useful admin tools to keep in the toolbox. 

```
#Find names of containers to stop
sudo docker container ls -a
#Stop a container identified by the code in the ls command
sudo docker stop container_id
#Remove a container identified by the code in the ls command
sudo docker rm container_id
```

That completes the setup of R, Shiny, the github link to Shiny, Docker and Grafana.